In [1]:
# -*- coding: utf-8 -*-
"""
@Author: Guan-Fu Liu
Created on Jan. 14, 2024
Last modified on Jan. 23, 2023

Convert the stellar yield file from Nomoto et al. (2013) to the format more convenient for 
further IMF determination.

Note that the AGB yields are net yields while the SNcc yields are total yields.
We must distinguish between these two kinds of yields.
"""
import numpy as np
import pandas as pd
import pyatomdb

In [2]:
yield_file = './Original/Nomoto2013.dat'
with open(yield_file, 'r') as f:
    lines = f.readlines()

z_line = []
for i in range(len(lines)):
    if "Z=" in lines[i]:
        E = float(lines[i+2].split()[1])
        if E < 10:
            z_line.append(("%s-SN" % lines[i].split()[1], i+1, np.minimum(len(lines[i+1]), 404)))
        else:
            z_line.append(("%s-HN" % lines[i].split()[1], i+1, np.minimum(len(lines[i+1]), 404)))
    else:
        pass


z_line = np.array(z_line, dtype=[('model', '<U20'), ('line', 'i4'), ('ncol', 'i4')])

# Some explantion

We discard the yields when Z=0.0000, $M_{\mathrm{ini}}\geq 140 M_{\odot}$, and $E_{51}\geq 9$ in the 
first part of the N13 yields.
Therefore, the `colspecs` goes as the following cell shows.

In [3]:
colspecs = [ ]
# A list of tuples giving the extents of the fixed-width fields of each line as half-open intervals (i.e., [from, to[ ).
# For Nomoto et al. (2013), the colspecs is as follows:
# 14, 13, 13, 13, ......, 13, 11+1('\n'),
for i in range(31):
    if i == 0:
        colspecs.append((0, 14))
    else:
        colspecs.append((14+13*(i-1), 14+13*i))

In [4]:
def get_df(model):
    """
    Get the dataframe from the yield_file.

    Parameters
    ----------
    model : str
        The model name.
        It is something like "0.0000-HN", "0.0000-SN", etc.
    Returns
    -------
    df : dataframe
    """
    try:
        j = np.where(z_line['model'] == model)[0][0]
    except:
        print("Cannot find the model %s" % model)
    
    colspecs = [ ]
    # A list of tuples giving the extents of the fixed-width fields of each line as half-open intervals (i.e., [from, to[ ).
    # For Nomoto et al. (2013), the colspecs is as follows:
    # 14, 13, 13, 13, ......, 13, 11+1('\n'),
    for i in range(int((z_line[j]['ncol']-1)/13)):
        if i == 0:
            colspecs.append((0, 14))
        else:
            colspecs.append((14+13*(i-1), 14+13*i))
    
    if j+1 < len(z_line):
        nrows = z_line[j+1][1]-z_line[j][1]-2
    else:
        nrows = None
    df = pd.read_fwf(yield_file, colspecs=colspecs, skiprows=z_line[j][1], nrows=nrows)
    df = df.loc[~df['M'].str.contains('E')]
    df = df.reset_index(drop=True)
    return df

In [5]:
def proc_df(model):
    """
    Process the dataframe from the get_df.

    Parameters
    ----------
    model : str
        The model name.
        It is something like "0.0000-HN", "0.0000-SN", etc.
    Returns
    -------
    df : dataframe
    """
    df = get_df(model)
    df_N13 = df.loc[~df['M'].str.contains('Mrem')].copy()
    # Convert mass to number
    df_N13.iloc[:, 1:] = df_N13.iloc[:, 1:].div(df_N13['M'].str.split().str[1].astype(float), axis=0)
    # Add element column
    df_N13.loc[:, 'element'] = df_N13['M'].str.split().str[0]
    # Drop M column
    df_N13.drop(columns=['M'], inplace=True)
    df_N13 = df_N13.set_index('element')
    df_N13.reset_index(inplace=True)
    # Sum up the yields of the same element
    df_N13 = df_N13.groupby('element').sum()
    df_N13.reset_index(inplace=True)
    # Sort the dataframe by the atomic number
    df_N13.loc[:, 'Z'] = df_N13['element'].apply(lambda x: pyatomdb.atomic.elsymb_to_Z(x))
    df_N13.sort_values(by='Z', inplace=True)
    df_N13.drop(columns=['Z'], inplace=True)
    df_N13.set_index('element', inplace=True)
    return df_N13

In [6]:
dfs = { }
solar_set = 'Lodders' # 'Default', 'AG', 'Allen', 'RA', 'Grevesse', 'GS', 'Lodders'
for model in z_line['model']:
    if 'SN' in model:
        df_tmp = proc_df(model)
        df_tmp.to_csv('./N13-all/N13-%s.csv' % model, float_format='%.4e')
        # Drop the cases where all the yields are zero,
        # which means they are SNIa yields.
        dfs[model] = df_tmp.loc[:, df_tmp.abs().sum(axis=0).values > 0]
        # AGB: Mini < 10 Msun
        dfs[model].loc[:, dfs[model].columns.astype(float)<10].to_csv('./AGB/N13-%s-AGB-net.csv' % model[:-3], float_format='%.4e')
        # SNcc: Mini > 10 Msun
        dfs[model].loc[:, dfs[model].columns.astype(float)>10].to_csv('./SNcc/N13-%s-SNcc-total.csv' % model[:-3], float_format='%.4e')

    else:
        df_tmp = proc_df(model)
        dfs[model] = df_tmp.loc[:, df_tmp.abs().sum(axis=0).values > 0]
        dfs[model].to_csv('./HN/N13-%s-HN-total.csv' % model[:-3], float_format='%.4e')
        # The float format of original yield files are %0.4e. 

In [7]:
z_line['model'][0][:-3]

'0.0000'